1. Encoding:
    - Encode train.csv as paragraph vectors, make sure they are shuffled, take half of the paragraph vectors as 'model'. Other half is 'training'. 
    - Encode test.csv as paragraph vectors, call them 'testing'.
2. Training:
    - For each training vector, compute similarity vector, which is PCA(training similarity with all model vectors). 
    - Train SVM to classify training vector class based on its similarity vector.
3. Inference:
    - For each testing vector, compute similarity vector just like in training, and use trained SVM to predict the class.

In [1]:
import torch
from sentence_transformers import SentenceTransformer
if torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
model = SentenceTransformer('all-MiniLM-L6-v2', device=device)
print("Model is loaded on:", model.device)

/Users/bensmus/Desktop/Projects/AG_news_classify/myenv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/bensmus/Desktop/Projects/AG_news_classify/myenv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Model is loaded on: mps:0


In [2]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle